In [3]:
import numpy as np
import pandas as pd

import twint

# Fixes runtime errors with twint
import nest_asyncio
nest_asyncio.apply()

import matplotlib
%matplotlib inline

In [4]:
def twint_search(search, username=None, since=None, until=None, limit=None):
    '''
    Function to return a pandas dataframe of tweets in English containing term using twint search method.
    Required parameter: search term.
    Optional parameters: username, beginning (since) and end (until) datetimes to search, maximum number of tweets.
    '''
    c = twint.Config()
    c.Lang = 'en'
    c.Search = search
    c.Username = username
    c.Since = since
    c.Until = until
    c.Limit = limit
    c.Pandas = True
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    return df

In [5]:
%time mask_df = twint_search(search='mask', username=None, since='2020-02-01', until='2020-02-02', limit=None)

1223757879305367552 2020-02-01 18:59:49 EDT <chong_wh96> Bought Bruno Fernandes, play him at CDM? Wtf are u thinking Ole. Its just like u buy a box of mask for Coronavirus, and u put them in store room.
1223757871751426049 2020-02-01 18:59:47 EDT <Dimplez> Because Nigeria is currently 51% Muslim and the GOP currently aiming to mask religious discrimination and xenaphobia under a guise of national security.  https://twitter.com/JesseLehrich/status/1223364387387342849 …
1223757866911326209 2020-02-01 18:59:46 EDT <Harry31348302> When you were a mask to stop the cronavirus but it says made in china  pic.twitter.com/HrsmEL5Neu
1223757863627018241 2020-02-01 18:59:45 EDT <Softdrink_117> This 'MeshGraphic' implementation supports:  - Input meshes of any size/complexity, including 3D - Control for Z-thickness - Masking from RectMask2D or Image Mask (if the shader used supports them!) - UV'd or UVless meshes (UVs will be auto-generated if the mesh lacks them)
1223757838071336960 2020-02-01 18:

1223756947683446784 2020-02-01 18:56:07 EDT <Catfeesh_Goree> Is that a velour face mask?
1223756946018164736 2020-02-01 18:56:06 EDT <Jfcdoomblade> Love the fact that Harley Quinn and Black Mask are in an R rated film in 2020  pic.twitter.com/VSnGhEYX2C
1223756931594096640 2020-02-01 18:56:03 EDT <JiveJimboFisher> Wear a face mask and disinfect
1223756891387547650 2020-02-01 18:55:53 EDT <Basement_Bum> Maybe it's just cold outside. It's common to wear a mask when it's cold outside. lol
1223756858969722880 2020-02-01 18:55:46 EDT <WannaBeCritic88> I have a Mask of the Phantasm one that’s very similar. Just incredible
1223756847737450497 2020-02-01 18:55:43 EDT <AlexisCT315> Had an INCREDIBLE time celebrating your birthday, Kynslee!! Braddock wants to wear his cape and mask to bed tonight 😂 danellgunter  https://www.instagram.com/p/B8CyCuTHUJTYOuYJnEcEUQzuu_pzRRhunFAisE0/?igshid=11uytyn68w0m5 …
1223756821170663425 2020-02-01 18:55:37 EDT <chercook11> This article is great info and so tru

1223755969449971712 2020-02-01 18:52:14 EDT <Wildstar64> Yes. Now have that nifty solder mask process that is cured under UV light.
1223755923417653248 2020-02-01 18:52:03 EDT <13thTrumpeter> bevel gone ✅, shoulder stripes gone ✅, Bringing back the white face mask ✅.  pic.twitter.com/HAAvNIIsNi
1223755907827408898 2020-02-01 18:51:59 EDT <osnapitzIvN> Oh forgot to mention the part where I had to MAKE AN EGG FACE MASK (with a real life egg) and put it on my face to stop the icthing :)))))) loves it🥚🍳  https://twitter.com/osnapitzIvN/status/1223755623331921920 …
1223755907433222144 2020-02-01 18:51:59 EDT <amindahaug1> Can we please get a Norwegian version of the masked singer? I want to be able to guess who’s behind the mask
1223755906451722243 2020-02-01 18:51:59 EDT <yournextbey> I would like to introduce the paradoxical thread that flows through the modern stressed college girl.   Life is falling apart but maybe this peel off charcoal face mask that advertises “being a magnet to impu

1223754970412937216 2020-02-01 18:48:15 EDT <EngineerBarbie> I am with you Stem! Bringing gloves for my flight, but have no mask. If he was on public transportation he could pass it to anyone.
1223754958035513344 2020-02-01 18:48:12 EDT <mbeomie> Yes ate. Para di sayang yung ticket. Wear a mask na lang then use alcohol.
1223754955615502336 2020-02-01 18:48:12 EDT <bfflino> subk tweet fr bc if i show up to unlock without a mask and they don’t let me do hi touch...
1223754949374414850 2020-02-01 18:48:10 EDT <bhealhty> VO2MAX Workout Training Mask | Oxygen Restricting High Elevation 16-Level Adjustable Altitude Resistance | Cardio Fitness Crossfit Cycling Running Triathlon Lifting Endurance Sports Exercise -  pic.twitter.com/UQjsio9ygj
1223754936917417984 2020-02-01 18:48:07 EDT <CrochetgodzillA> Monster with Mask 1H-Digital by Sabby World of Creations on @Zibbet https://marketplace.zibbet.com/sabby-world-of-creations/monster-with-mask-1h-digital-kit-jewelry-tag-clipart-gift-tag-holiday-

1223754193409839104 2020-02-01 18:45:10 EDT <Smudge64> Please don't be fuelled by fear with all of the media attention around this new corona virus strain, and don't be an arsehole to someone choosing to wear a surgical mask.  https://www.instagram.com/p/B8Cw1NelwzP/?igshid=1ttx7893ga2sn …
1223754187181158401 2020-02-01 18:45:09 EDT <pj_campbell> I’m with you 100% on that. I like him a ton in Mask of Zorro. To be honest, that whole storyline is the weakest part of the movie by far. Rene and Mel are great, and so is Glover. But the villain stuff doesn’t work for me much.
1223754168256483328 2020-02-01 18:45:04 EDT <kenvanmetre> You have 2 pictures. One has two guys in masks from the front. The other nobody wearing a mask taken from the back. There is no news here, just some asshole with a camera try to scare you.
1223754163739193345 2020-02-01 18:45:03 EDT <TheRaveStop> That mask👀👽
1223754160496988160 2020-02-01 18:45:02 EDT <Dermatology_bio> 20172025 World Facial Mask Market Research R

...

1223473456999620608 2020-02-01 00:09:37 EDT <emtinson> scuba training isn’t cheap but it’s definitely attainable, corinth! the mask gets in the way of some activities though
1223473449055784960 2020-02-01 00:09:35 EDT <mplngr> Quote of the day: “Zhan said people could still go for a walk in the park, but they should avoid group dance activities... it will be hard to breathe with a mask on while you’re dancing”. #GoodPoint #ThingsIveNeverThoughtAbout #2019nCoV #coronarvirus https://twitter.com/Laurie_Garrett/status/1223426726732226562 …
1223473431192076288 2020-02-01 00:09:31 EDT <babyyykylee> AND im making my own dust mask 🥴 crazyyyt
1223473427081617409 2020-02-01 00:09:30 EDT <eroticapjm> always wear mask
1223473425009778688 2020-02-01 00:09:30 EDT <dyk5s> i don't get how hard it is for y'all to understand that she isnt bothered by the mask and no one is forcing her.. you got a a bunch of idiots making tweets "joking" about how shes forced to be in the mask and making other idiots bel

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 27.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 64.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 125.0 secs


1223473160865157121 2020-02-01 00:08:27 EDT <vaintshit> i swear people wait till i put my face mask on to ft me. like bitch im not here 😭
1223473135892041729 2020-02-01 00:08:21 EDT <naz_grgl> Air-travel checklist: - PM 2.5 mask - diving mask - O2 tanks  Did I forget any essentials? 😆  pic.twitter.com/p2877BPqvK
1223473130363998210 2020-02-01 00:08:19 EDT <bunnydream9788> And of course with mask 😬  pic.twitter.com/cpmfSEzyRL
1223473111208812544 2020-02-01 00:08:15 EDT <DCuplink> YOU CITIZEN, PICK UP THAT CAN (AND PUT YOUR MASK ON)
1223473096113278977 2020-02-01 00:08:11 EDT <PeaceLoveCarol> Ima bedazzle my mask.
1223473067994775552 2020-02-01 00:08:05 EDT <Honduran_Flower> Wow this explains what I saw this morning🤔 damn I am going to start wearing a mask. Sounds like its getting serious.  pic.twitter.com/byVrYeqRoJ
1223473059669139456 2020-02-01 00:08:03 EDT <_ARGET_> This is a common misconception,  there is only ONE way of wearing a face/surgical mask, the blue one outside.  kaya wag

1223472062921986048 2020-02-01 00:04:05 EDT <CarlBowers1> Says the punk hiding behind a mask.
1223472049126961152 2020-02-01 00:04:02 EDT <mnakagawa1212> When I was with someone right next to me, I had a mask 😷 on and coughed once.  The guy said “Are you sick of Coronavirus?”  I was like “Seriously!?”  That was the moment I noticed infection is being more serious that I thought...
1223472045113184257 2020-02-01 00:04:01 EDT <DNicole2314> Nothing better than babies in bed , hot bubble bath , face mask and wine 🍷 🙃
1223472044299313152 2020-02-01 00:04:01 EDT <ArthurE69944934> because we’re short of mask
1223472015543128064 2020-02-01 00:03:54 EDT <belxrosee> I've been seeing people wear their face mask the wrong way, and it saddens me. Cause 1- I know they just want to protect themselves. And 2- Face mask are overpriced and OOS almost everywhere 😢. But, I'm always shy to approach random strangers and be that "know it all" person...
1223472014167379968 2020-02-01 00:03:53 EDT <ssabeell> F

CPU times: user 5min 5s, sys: 4.22 s, total: 5min 9s
Wall time: 19min 8s


In [6]:
len(mask_df)

14516

In [7]:
mask_df.keys()

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username',
       'name', 'day', 'hour', 'link', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [8]:
mask_df.head()

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1223757879305367552,1223757879305367552,1580601589000,2020-02-01 18:59:49,EDT,,"Bought Bruno Fernandes, play him at CDM? Wtf a...",[],[],2221552537,...,,,,,,"[{'user_id': '2221552537', 'username': 'chong_...",,,,
1,1223757871751426049,1223757871751426049,1580601587000,2020-02-01 18:59:47,EDT,,Because Nigeria is currently 51% Muslim and th...,[],[],21841872,...,,,,,,"[{'user_id': '21841872', 'username': 'Dimplez'}]",,,,
2,1223757866911326209,1223757866911326209,1580601586000,2020-02-01 18:59:46,EDT,,When you were a mask to stop the cronavirus bu...,[],[],1209988745790918656,...,,,,,,"[{'user_id': '1209988745790918656', 'username'...",,,,
3,1223757863627018241,1223756689301594112,1580601585000,2020-02-01 18:59:45,EDT,,This 'MeshGraphic' implementation supports:\n\...,[],[],1478940204,...,,,,,,"[{'user_id': '1478940204', 'username': 'Softdr...",,,,
4,1223757838071336960,1223718724240203776,1580601579000,2020-02-01 18:59:39,EDT,,Chill bruh!! Did i step on your balls or somet...,[],[],1097901450204794880,...,,,,,,"[{'user_id': '1097901450204794880', 'username'...",,,,


In [10]:
len(mask_df['date'].unique())

13061

In [15]:
mask_df['date'][0][0:10]

'2020-02-01'

In [17]:
mask_df['date'][0][11:]

'18:59:49'